In [ ]:
import cv2
#from IPython.display import display
import numpy as np
# import ipywidgets
from ipywidgets import interact, interactive, fixed
from skimage import filters, io
from matplotlib import pyplot as plt

In [ ]:
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [ ]:
cv2.__version__

In [ ]:
img = cv2.imread('house.png')

In [ ]:
type(img)

In [ ]:
# opencv returns BGR; convert to RGB for display using matplotlib
img2 = img[:,:,::-1]
#io.imshow(img2)
plt.imshow(img2)

In [ ]:
img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#display(io.Image(img_grey))
#io.imshow(img_grey)
plt.imshow(img_grey, cmap='gray')

In [ ]:
# the object containing a grey image (after convertion) is an integer array
# see more about conversion at
# http://docs.opencv.org/modules/imgproc/doc/miscellaneous_transformations.html

img_grey[1][1], type(img_grey)

In [ ]:
img_grey.size, img_grey.itemsize, img_grey.shape, img_grey.dtype

In [ ]:
# convert image to floats either using dtype
#newimg = np.ndarray(shape=img_grey.shape, dtype=float)
# or by dividing the original array by max value

newimg = img_grey/255.

In [ ]:
newimg[1][1], type(img_grey)

In [ ]:
newimg.size, newimg.itemsize, newimg.shape, newimg.dtype

In [ ]:
# convert to float and show a segment only
#img_grey *= 1./255.
#display(io.Image(newimg))
plt.imshow(newimg[:200,:200], cmap='gray')

### DoG

In [ ]:
## DoG filter.
#  @param  img      input gray image.
#  @param  sigma    sigma for small Gaussian filter.
#  @param  k_sigma  large/small sigma (Gaussian filter).
def DoG(img, s, k):
    sL = s * k
    G_small = cv2.GaussianBlur(img,(5, 5), sigmaX=s,  sigmaY=s)
    G_large = cv2.GaussianBlur(img,(5, 5), sigmaX=sL, sigmaY=sL)
    D = G_small - G_large
    return D

In [ ]:
## DoG edge detection.
#  @param  img      input gray image.
#  @param  sigma    sigma for small Gaussian filter.
#  @param  k_sigma  large/small sigma (Gaussian filter).
#  @param  epsilon  threshold value for edge detection.
def DoGEdge(img, s, k, eps):
    D = DoG(img, s, k)
    retval, D_edge = cv2.threshold(D, eps, 1.0, cv2.THRESH_BINARY)
    return D_edge

In [ ]:
img_dog =DoG(newimg, 1.0, 2.0)

In [ ]:
A = cv2.GaussianBlur(newimg,(5, 5), sigmaX=1.0,  sigmaY=1.0)
B = cv2.GaussianBlur(newimg,(5, 5), sigmaX=2.0,  sigmaY=2.0)
C = A - B
A[1][1], B[1][1], C[1][1]

In [ ]:
def showImg(s=1.0, k=2.0):
    #new_image = io.Image(DoG(img_grey, s, k))
    new_image = plt.imshow(DoG(img_grey, s, k), cmap='gray')
    return new_image

In [ ]:
w = interactive(showImg, 
         s=(0.1, 10.0, 0.1), 
         k=(0.1, 10.0, 0.1))
display(w)

### XDoG

In [ ]:
## Sharp image from scaled DoG signal.
#  @param  img        input gray image.
#  @param  sigma      sigma for small Gaussian filter.
#  @param  k_sigma    large/small sigma (Gaussian filter).
#  @param  p          scale parameter for DoG signal to make sharp.
def sharpImage(img, s, k, p):
    sL = s * k
    G_small = cv2.GaussianBlur(img,(5, 5), sigmaX=s,  sigmaY=s)
    G_large = cv2.GaussianBlur(img,(5, 5), sigmaX=sL, sigmaY=sL)
    S = (1+p) * G_small - p * G_large
    return S

In [ ]:
## Soft threshold function to make ink rendering style.
#  @param  img        input gray image.
#  @param  epsilon    threshold value between dark and bright.
#  @param  phi        soft thresholding parameter.
def softThreshold(SI, eps, phi):
    T = np.zeros(SI.shape)
    SI_bright = SI >= eps
    SI_dark = SI < eps
    T[SI_bright] = 1.0
    T[SI_dark] = 1.0 + np.tanh( phi * (SI[SI_dark] - eps))
    #T[SI_bright] = 255
    #T[SI_dark] = min(max(round( 255*(1.0 + np.tanh( phi * (SI[SI_dark] - eps)) ))))
    return T

In [ ]:
## XDoG filter.
#  @param  img        input gray image.
#  @param  sigma      sigma for sharpImage.
#  @param  k_sigma    large/small sigma for sharpImage.
#  @param  p          scale parameter for sharpImage.
#  @param  epsilon    threshold value for softThreshold.
#  @param  phi        soft thresholding parameter for softThreshold.
def XDoG(img, s, k, p, eps, phi):
    S = sharpImage(img, s, k, p)
    SI = np.multiply(img, S)
    T = softThreshold(SI, eps, phi)
    return T

In [ ]:
img_xdog =XDoG(img_grey, 1.2, 2.0, 15.0, 0.5, 2.0)

In [ ]:
#display(io.Image(img_xdog))
plt.imshow(img_xdog, cmap='gray')

In [ ]:
def showImgXDoG(s=1.2, k=2.0, p=15.0, eps=0.5, phi=2.0):
    #new_image = io.Image(XDoG(img_grey, s, k, p, eps, phi))
    new_image = plt.imshow(XDoG(img_grey, s, k, p, eps, phi), cmap='gray')
    return new_image

In [ ]:
interactive(showImgXDoG, 
         s=(0.1, 10.0, 0.1), 
         k=(0.1, 10.0, 0.1),
         p=(0.1, 100.0, 0.1),
         eps=(-10.0, 10.0, 0.1),
         phi=(0.1, 100.0, 0.1) )